In [1]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.4 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [2]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [5]:
model = pyo.ConcreteModel()

model.i = pyo.Set(initialize = ['Plant11', 'Plant12', 'Plant13', 'Plant14',
                                'Plant21', 'Plant22', 'Plant23', 'Plant24',
                                'Plant31', 'Plant32', 'Plant33', 'Plant34',])

# Sets
model.i = pyo.RangeSet(1, 3)
model.j = pyo.RangeSet(1, 4)

#Parameters

model.S = pyo.Param(model.i, initialize = {1: 35, 2:50, 3: 40})
S = model.S

model.D = pyo.Param(model.j, initialize = {1: 45, 2: 20, 3:30, 4: 30})
D = model.D

model.P = pyo.Param(model.i, model.j, initialize = {(1, 1): 8, (1,2): 6, (1,3): 10, (1,4): 9,
                                                    (2, 1): 9, (2,2): 12, (2,3): 13, (2,4): 7,
                                                    (3, 1): 14, (3,2): 9, (3,3): 16, (3,4): 5,
                                                    })

P = model.P

model.x = pyo.Var(model.i, model.j, within = pyo.NonNegativeReals)
x = model.x

def Objective_rule(model):
  return sum(sum(model.P[i, j]*model.x[i, j] for i in model.i) for j in model.j)

model.Obj = pyo.Objective(rule = Objective_rule, sense = pyo.minimize)

def ConstraintS1(model, i):
  return sum(x[1, j] for j in model.j) <= S[1]

model.Const1 = pyo.Constraint(model.j, rule = ConstraintS1)

def ConstraintS2(model, i):
  return sum(x[2, j] for j in model.j) <= S[2]

model.ConstS4 = pyo.Constraint(model.j, rule = ConstraintS2)

def ConstraintS3(model, i):
  return sum(x[3, j] for j in model.j) <= S[3]

model.ConstS3 = pyo.Constraint(model.j, rule = ConstraintS3)

def ConstraintD1(model, j):
  return sum(x[i, 1] for i in model.i) >= D[1]

model.ConstD1 = pyo.Constraint(model.i, rule = ConstraintD1)

def ConstraintD2(model, j):
  return sum(x[i, 2] for i in model.i) >= D[2]

model.ConstD2 = pyo.Constraint(model.i, rule = ConstraintD2)

def ConstraintD3(model, j):
  return sum(x[i, 3] for i in model.i) >= D[3]

model.ConstD3 = pyo.Constraint(model.i, rule = ConstraintD3)

def ConstraintD4(model, j):
  return sum(x[i, 4] for i in model.i) >= D[4]

model.ConstD4 = pyo.Constraint(model.i, rule = ConstraintD4)


Solver = SolverFactory('glpk')

results = Solver.solve(model)

print(results)

print("Objective Function", model.Obj())

for i in model.i:
  for j in model.j:
    print('Electricity send from Plant', i, 'to city', j, '=', x[i, j]())

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().



Problem: 
- Name: unknown
  Lower bound: 1020.0
  Upper bound: 1020.0
  Number of objectives: 1
  Number of constraints: 24
  Number of variables: 12
  Number of nonzeros: 84
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.019538164138793945
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective Function 1020.0
Electricity send from Plant 1 to city 1 = 0.0
Electricity send from Plant 1 to city 2 = 10.0
Electricity send from Plant 1 to city 3 = 25.0
Electricity send from Plant 1 to city 4 = 0.0
Electricity send from Plant 2 to city 1 = 45.0
Electricity send from Plant 2 to city 2 = 0.0
Electricity send from Plant 2 to city 3 = 5.0
Electricity send from Plant 2 to city 4 = 0.0
Electricity send from Plant 3 to city 1 = 0.0
Electricity send from Plant 3 to city 2 = 10.0
Electricity send from Plant 3 